In [1]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationSummaryMemory
# from langchain.chains import ConversationalRetrievalChain
import os
from dotenv import load_dotenv
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage

## get keys

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.environ.get('GOOGLE_API_KEY')

## clone an repository

In [2]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/Sumegh20/myIPYNBrenderer.git", to_path=repo_path)

<git.repo.base.Repo '/home/sumgh/Data Science/projects/code-analyzer/research/test_repo/.git'>

## Load data 

In [6]:
loader = GenericLoader.from_filesystem(path="test_repo",
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()

## chunking

#### Note: RecursiveCharacterTextSplitter does not support multiple languages in one shot. For this, we need to write a custom RecursiveCharacterTextSplitter class.

In [12]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 1000,
                                                             chunk_overlap = 200)

text_splitter = documents_splitter.split_documents(documents)

## Embdding

In [4]:
embeddings=OpenAIEmbeddings(disallowed_special=())

/home/sumgh/Data Science/projects/code-analyzer/code_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Create VectorDB

In [8]:
db_path = "faiss_index"
vectordb = FAISS.from_documents(text_splitter, embeddings)
vectordb.save_local(db_path)

In [5]:
kn_base = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [6]:
retriever=kn_base.as_retriever(search_type="mmr", search_kwargs={"k":3})

## Prompt

In [7]:
template="""You are an assistant for question-answering tasks while prioritizing a seamless user experience.
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use five sentences maximum and keep the answer concise.
            You should be able to remember and reference the last three conversations between you and the user.
            Maintain a friendly, positive, and professional tone throughout interactions.
            Question: {question}
            Context: {context}
            Chat history: {chat_history}
            Answer:
        """
prompt=ChatPromptTemplate.from_template(template)


## LLM

In [8]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) #, streaming=True)

/home/sumgh/Data Science/projects/code-analyzer/code_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## RAG chain

In [15]:
def input_handeler(input: dict):
    return input['question']

In [16]:
rag_chain = (
            # {"context": retriever,  "question": RunnablePassthrough()}
            RunnablePassthrough().assign(
                context = input_handeler | retriever
            )
            | prompt
            | llm
            | StrOutputParser()
        )

In [17]:
chat_history = []

In [24]:
question = "What was my second previous question?"
responce = rag_chain.invoke({"question": question, "chat_history":chat_history})

print(responce)

Your second previous question was about explaining the render_site function.


In [22]:
chat_history.extend(
    [
        HumanMessage(
            role="user", 
            content=question
        ),
        AIMessage(
            role="assistant",
            content=responce
        ),
    ]  
)

## Final Pipeline

In [1]:
from constant import file_extensions, file_language

In [2]:
language = 'PYTHON'
file_language.get(language, None)

<Language.PYTHON: 'python'>

In [ ]:
#clone any github repositories 
def repo_ingestion(repo_url):
    os.makedirs("repo", exist_ok=True)
    repo_path = "repo/"
    Repo.clone_from(repo_url, to_path=repo_path)


# Create suffixes from language
def get_suffixes_from_language(language):
    suffixes = file_extensions.get(language, None)
    return suffixes


def get_language_from_language(language):
    Languages = file_language.get(language, None)
    return Languages


#Loading repositories as documents
def create_document_from_repo(repo_path, language):
    suffixes = get_suffixes_from_language(language)
    Languages = get_language_from_language(language)

    loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=suffixes,
                                       parser = LanguageParser(language=Languages, parser_threshold=500)
                                        )
    documents = loader.load()
    return documents


#Creating text chunks 
def text_splitter(documents):
    documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 1000,
                                                             chunk_overlap = 200)
    
    text_chunks = documents_splitter.split_documents(documents)
    return text_chunks


#loading embeddings model
def load_embedding():
    embeddings=OpenAIEmbeddings(disallowed_special=())
    return embeddings


#creating knowlagebase
def create_knowledgebase(texts, db_path="faiss_index"):
    embeddings = load_embedding()

    vectordb = FAISS.from_documents(texts, embeddings)
    vectordb.save_local(db_path)


#loading knowledgebase
def get_knowledge_base(db_path, embedding):
    vectordb = FAISS.load_local(db_path, embedding, allow_dangerous_deserialization=True)

    return vectordb


#Create Prompt
def create_prompt():
    template="""You are an assistant for question-answering tasks while prioritizing a seamless user experience.
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use five sentences maximum and keep the answer concise.
            You should be able to remember and reference the last three conversations between you and the user.
            Maintain a friendly, positive, and professional tone throughout interactions.
            Question: {question}
            Context: {context}
            Chat history: {chat_history}
            Answer:
        """
    prompt=ChatPromptTemplate.from_template(template)

    return prompt


# input handeler
def input_handeler(input: dict):
    return input['question']


# create RAG chain
def create_rag_chain(llm, retriever, prompt):
    rag_chain = (
            # {"context": retriever,  "question": RunnablePassthrough()}
            RunnablePassthrough().assign(
                context = input_handeler | retriever
            )
            | prompt
            | llm
            | StrOutputParser()
        )
    
    return rag_chain


def get_llm():
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True)

    return llm

#user_input
def get_output_stream(question, chat_history):
    embedding = load_embedding()
    vectordb = get_knowledge_base(db_path=database_path, embedding=embedding)
    llm = get_llm()
    retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3})
    prompt = create_prompt()

    rag_chain = create_rag_chain(llm=llm, retriever=retriever, prompt=prompt)
    
    result = rag_chain.stream({"question": question, "chat_history":chat_history})
    return result

In [ ]:
import requests

def is_github_url_valid(url):
    try:
        response = requests.head(url)
        if response.status_code == 200:
            return True
        else:
            return False
    except requests.exceptions.RequestException:
        return False

# Example usage
github_url = "https://github.com/alferov/awesome-gulp"
if is_github_url_valid(github_url):
    print(f"{github_url} is a valid GitHub URL.")
else:
    print(f"{github_url} is not a valid GitHub URL.")